# Anticipez les besoins en consommation électrique de bâtiments

- **Projet 4 du parcours « Data Scientist » d’OpenClassrooms**
- **Mark Creasey**

## Partie 2 : Modélisation et Résultats

<img height="64" src="https://user.oc-static.com/upload/2019/02/24/15510245026714_Seattle_logo_landscape_blue-black.png" alt="Logo seattle">


## 1.1 Mission

À partir des [relevés déjà réalisés en 2015 et 2016](https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking#2015-building-energy-benchmarking.csv) :

- **prédire les émissions de CO2** et la **consommation totale d’énergie** de bâtiments commerciales en Seattle pour lesquels elles n’ont pas encore été mesurées, basé sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction ...).

- **évaluer l’intérêt de l’[ENERGY STAR Score](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/interpret-your-results/what) pour la prédiction d’émissions**.


## 1.2 Requirements : Bibliothèques utilisées dans ce notebook


## 1.3 Des fonctions utilitaires


## 1.4 Métriques d'évaluation


# 2. Données d'entrainement et test


# 3. Prédiction de la consommation énergétique


## Modèle baseline (dummyRegressor)


## Modèle linéaire


## Modèle non-linéaire


## Modèle ensembliste


## Évaluation des prédictions (analyse de erreurs)


## Comparaison et choix du modèle


# 4. Prédiction des émissions CO2


# 5. Influence de l'ENERGYSTARScore sur les prédictions


# 6. Résultats

## Comparaison des performances des modèles


# 7. Conclusion
